In [5]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from enum import Enum
from pydantic import BaseModel, Field

In [6]:
import getpass
import os
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [ ]:

from typing_extensions import TypedDict
from typing import Annotated, Literal , List
from pydantic import BaseModel, Field , field_validator, ValidationInfo, model_validator
from langgraph.graph.message import AnyMessage , add_messages
from langgraph.graph import StateGraph, MessagesState, START, END
import sys
import os
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "../..")))

from booking_agent.api.booking import BookingAPI
from booking_agent.api.geoCoding import GeoCodingAPI
from booking_agent.api.getKey import OAuthClient
from booking_agent.api.getQuotes import QuotesAPI
from booking_agent.api.is_Airport import IsAirport
jupiterAPI = os.getenv('JUPITER_API')
quoteAPI = str(jupiterAPI) + "/demand/v1/quotes"
bookingsAPI  = str(jupiterAPI) + '/demand/v1/bookings'

class BookingCarDetails(BaseModel):
    """Details for the bookings car details"""
    name: str = Field(
        ...,
        description="The name of the person booking the ride. Do not autofill if not provided",
    )
    number_phone: str = Field(
        ...,
        description="The phone number of the user. Do not autofill if not provided",
    )
    pick_up_location: str = Field(
        ...,
        description="The location where the user will be picked up. This can be a full address or a specific location name. Do not autofill if not provided",
    )
    destination_location: str = Field(
        ...,
        description="The destination location for the ride. This can be a full address or a specific location name. Do not autofill if not provided"
    )
    pick_up_time: str = Field(
        ...,
        description="The time the user intends to be picked up. No format keeps the text related to time. Do not autofill if not provided"
    )
    flight_code: str = Field(
        # default= 'None',
        ...,
        description="Flight numbers, consisting of letters and numbers, usually start with the airline code (e.g. VN123, SQ318)."
    )
    
    @field_validator('pick_up_location')
    @classmethod
    def validate_pickup(cls, value:str):
        geoCodingAPI = GeoCodingAPI()
        if value == '':
            return ''
        else :
            geoCoding_pickup = geoCodingAPI.get_geocoding(value)
            if geoCoding_pickup["status"] == "OK" :
                return geoCoding_pickup['results'][0]['formatted_address']
            else:
                raise ValueError(f"Invalid pick-up location: {value}")
            
    @field_validator('destination_location')
    @classmethod
    def validate_destination(cls, value : str, info: ValidationInfo):
        geoCodingAPI = GeoCodingAPI()
        
        # print (geoCoding_destination['results'][0]['formatted_address'])
        if value == '':
            return ''
        else :
            geoCoding_destination = geoCodingAPI.get_geocoding(value)
            if geoCoding_destination["status"] == "OK":
                if geoCoding_destination['results'][0]['formatted_address'] == info.data['pick_up_location']:
                    raise ValueError(f"Invalid destination location: {value}")
                else:
                    return geoCoding_destination['results'][0]['formatted_address']
            else:
            
                raise ValueError(f"Invalid destination location: {value}")
    @model_validator(mode="after")
    def set_flight_code_if_airport(self):
        geoCodingAPI = GeoCodingAPI()
        API_Airport = IsAirport(base_url=jupiterAPI + '/v2/distance/airport')

        if self.pick_up_location:
            geoCoding_pickup = geoCodingAPI.get_geocoding(self.pick_up_location)
            if geoCoding_pickup["status"] == "OK":
                pick_up_lat = geoCoding_pickup['results'][0]['geometry']['location']['lat']
                pick_up_lng = geoCoding_pickup['results'][0]['geometry']['location']['lng']
                
                is_Airport = API_Airport.is_Airport(pick_up_lat, pick_up_lng)

                if is_Airport[0] == False:  # Nếu là sân bay
                    self.flight_code = 'No Request'
        
        return self


In [32]:
chain = llm.with_structured_output(BookingCarDetails)

In [38]:
reponse = chain.invoke("i want cancel")

In [40]:
reponse

BookingCarDetails(name='', number_phone='', pick_up_location='', destination_location='', pick_up_time='', flight_code='')

In [36]:
reponse = chain.invoke("i want change to pick up from Danang Airport at 9 tomorrow ")

In [37]:
reponse

BookingCarDetails(name='', number_phone='', pick_up_location='Danang International Airport, Đ. Nguyễn Văn Linh, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='', pick_up_time='9 tomorrow', flight_code='')

In [12]:
def check_what_is_empty(user_peronal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_peronal_details.model_dump().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(f"Field '{field}' is empty.")
            ask_for.append(f'{field}')
    return ask_for

In [13]:
ask_for = check_what_is_empty(reponse)
ask_for

Field 'name' is empty.
Field 'number_phone' is empty.
Field 'flight_code' is empty.


['name', 'number_phone', 'flight_code']

In [14]:
def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details

In [15]:
booking_details = BookingCarDetails(name="",number_phone="",pick_up_location="",destination_location="", pick_up_time="")

ValidationError: 1 validation error for BookingCarDetails
flight_code
  Field required [type=missing, input_value={'name': '', 'number_phon... '', 'pick_up_time': ''}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
booking_details = add_non_empty_details(booking_details ,reponse)

In [ ]:
booking_details

BookingCarDetails(name='', number_phone='', pick_up_location='460 Tran Dai Nghia, Da Nang', destination_location='271 Nguyen Van Linh, Da Nang', pick_up_time='9 tomorrow')

In [ ]:
ask_for = check_what_is_empty(booking_details)
ask_for

Field 'name' is empty.
Field 'number_phone' is empty.


['name', 'number_phone']

In [ ]:
def ask_for_info(ask_list:list):
    # prompt template 1
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n \
        ### ask_for list: {ask_for}"
    )
    # info_gathering_chain
    info_gathering_chain = first_prompt | llm | StrOutputParser()
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_list})
    print(first_prompt)
    return ai_chat

In [ ]:
def filter_response(text_input, user_details ):
    chain = llm.with_structured_output(BookingCarDetails)
    res = chain.invoke(text_input)
    # add filtered info to the
    user_details = add_non_empty_details(user_details,res)
    print(user_details)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for

In [ ]:
booking_details

NameError: name 'booking_details' is not defined

In [ ]:
ask_for

['name', 'number_phone']

In [ ]:
ask_for_info(ask_for)

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]


'I need to gather some information from you. Could you please tell me your name?'

In [ ]:
text_input ="ok My name is Sam"

In [ ]:
user_details, ask_for = filter_response(text_input, booking_details)

NameError: name 'booking_details' is not defined

In [ ]:
user_details

BookingCarDetails(name='', number_phone='0917181880', pick_up_location='460 Tran Dai Nghia, Da Nang', destination_location='271 Nguyen Van Linh, Da Nang', pick_up_time='9 tomorrow')

In [ ]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]
I need to gather some information from you. Could you please provide your phone number?


In [ ]:
text_input ="ok My name is Sam"
user_details, ask_for = filter_response(text_input, user_details)

name='Sam' number_phone='0917181880' pick_up_location='460 Tran Dai Nghia, Da Nang' destination_location='271 Nguyen Van Linh, Da Nang' pick_up_time='9 tomorrow'


In [ ]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

Everything gathered move to next phase


In [ ]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]
I need to gather some information from you. Could you please tell me your name?


In [ ]:
import requests
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field 
from langchain_core.tools import tool
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

class BookingCarDetails(BaseModel):
    """Details for the bookings car details"""
    name: str = Field(
        ...,
        description="The name of the person booking the ride.This is optional if provided",
    )
    number_phone: str = Field(
        ...,
        description="The phone number of the user.This is optional if provided",
    )
    pick_up_location: str = Field(
        ...,
        description="The location where the user will be picked up. This can be a full address or a specific location name.This is optional if provided",
    )
    destination_location: str = Field(
        ...,
        description="The destination location for the ride. This can be a full address or a specific location name.This is optional if provided"
    )
    pick_up_time: str = Field(
        ...,
        description="The time the user intends to be picked up. No format keeps the text related to time..This is optional if provided"
    )
    # @feild_validator('')


def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_personal_details.model_dump().items():
        if value in [None, "", 0]:  # Add other 'empty' conditions if needed
            print(f"Field '{field}' is empty.")
            ask_for.append(field)
    return ask_for


def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details


def ask_for_info(ask_list: list):
    first_prompt = ChatPromptTemplate.from_template(
        """Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. 
        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.
        ### ask_for list: {ask_for}"""
    )

    info_gathering_chain = first_prompt | llm | StrOutputParser()
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_list})
    print(first_prompt)
    return ai_chat
def filter_response(text_input, user_details ):
    chain = llm.with_structured_output(BookingCarDetails)
    res = chain.invoke(text_input)
    # add filtered info to the
    user_details = add_non_empty_details(user_details,res)
    print(user_details)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for
def ask_confirm_info(booking_details: BookingCarDetails):
    # booking_details.
    message = (
        f"Please confirm your ride details:\n"
        f"- Pickup Location: {booking_details.pick_up_location}\n"
        f"- Destination: {booking_details.destination_location}\n"
        f"- Pickup Time: {booking_details.pick_up_time}\n"
        f"- Name: {booking_details.name}\n"
        f"- Contact Number: {booking_details.number_phone}\n"
    )
    print(message)
@tool
def get_booking_details(input_text):
    """ This is function to get information for booking"""
    chain = llm.with_structured_output(BookingCarDetails)
    response_text = "i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow "
    response = chain.invoke(input_text)
    booking_details = BookingCarDetails(
        name="", number_phone="", pick_up_location="", destination_location="", pick_up_time=""
    )
    booking_details = add_non_empty_details(booking_details, response)
    
    ask_for = check_what_is_empty(booking_details)
    
    ai_response = ask_for_info(ask_for)
    print(ai_response)
    text_input = input()
    user_details, ask_for = filter_response(text_input, booking_details)
    while ask_for:  
        ai_response = ask_for_info(ask_for)
        input
        print(ai_response)
        text_input = input()
        user_details, ask_for = filter_response(text_input, user_details)
        print(ask_for)
    
    ask_confirm_info(user_details)
    return 
    
        

    # # Validate locations
    # validation_errors = validate_locations(booking_details)
    # if validation_errors:
    #     print(f"Validation errors: {validation_errors}")
    # else:
    #     print("All details are valid.")





In [ ]:

# chain = llm.with_structured_output(BookingCarDetails)
response = "i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow "
# response = chain.invoke(response_text)

get_booking_details(response)

Field 'name' is empty.
Field 'number_phone' is empty.
input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering some information to help you better. Could you please provide your name?


KeyboardInterrupt: 

In [ ]:
tools = [get_booking_details]

In [ ]:
from langgraph.prebuilt import create_react_agent

# system_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are very powerful assistant, but don't know current events",
#         ),
#         ("user", "{input}")
#     ]
# )
system_prompt = """
    You are a very powerful assistant. Be polite, clear, and understandable.
    If users ask general questions, answer them helpfully. 
    If users want to book a ride, call the 'get_booking_details' function to gather booking information.
    Please confirm information after done 'get_booking_details' .
    If user confirm, call the 'get_quotes' ,else thank you user
    
    """
agent_executor = create_react_agent(llm, tools = [get_booking_details] , state_modifier=system_prompt)

NameError: name 'llm' is not defined

In [ ]:
inputs = {"messages": [("user", "my name is Huy,I want to book a car from 271 Nguyen Van Linh to 466 NGuyen van linh da nang at now")]}
for s in agent_executor.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

my name is Huy,I want to book a car from 271 Nguyen Van Linh to 466 NGuyen van linh da nang at now
================================== Ai Message ==================================
Tool Calls:
  get_booking_details (call_u4Kd8HniyFL3uqhfphrCoolg)
 Call ID: call_u4Kd8HniyFL3uqhfphrCoolg
  Args:
    input_text: I want to book a car from 271 Nguyen Van Linh to 466 Nguyen Van Linh Da Nang at now.
Field 'name' is empty.
Field 'number_phone' is empty.
input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), a

In [ ]:
from icecream import ic

In [ ]:
pip install icecream

  Using cached executing-2.1.0-py2.py3-none-any.whl.metadata (8.9 kB)
Using cached executing-2.1.0-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: executing
    Found existing installation: executing 2.0.1
    Uninstalling executing-2.0.1:
      Successfully uninstalled executing-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from typing import Literal
from langgraph.graph import END


async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["agent", END],
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

NameError: name 'PlanExecute' is not defined

In [ ]:
import os
import requests
from typing import Annotated, Literal , List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field , field_validator, ValidationInfo
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langgraph.graph.message import AnyMessage
from API.booking import BookingAPI
from API.geoCoding import GeoCodingAPI
from API.getKey import OAuthClient
from API.getQuotes import QuotesAPI
from API.is_Airport import IsAirport

from langgraph.graph import StateGraph, MessagesState, START, END

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
jupiterAPI = os.getenv('JUPITER_API')
quoteAPI = str(jupiterAPI) + "/demand/v1/quotes"
bookingsAPI  = str(jupiterAPI) + '/demand/v1/bookings'

class BookingCarDetails(BaseModel):
    """Details for the bookings car details"""
    name: str = Field(
        ...,
        description="The name of the person booking the ride.This is optional if provided",
    )
    number_phone: str = Field(
        ...,
        description="The phone number of the user.This is optional if provided",
    )
    pick_up_location: str = Field(
        ...,
        description="The location where the user will be picked up. This can be a full address or a specific location name.This is optional if provided",
    )
    destination_location: str = Field(
        ...,
        description="The destination location for the ride. This can be a full address or a specific location name.This is optional if provided"
    )
    pick_up_time: str = Field(
        ...,
        description="The time the user intends to be picked up. No format keeps the text related to time..This is optional if provided"
    )
    @field_validator('pick_up_location')
    @classmethod
    def validate_pickup(cls, value:str):
        geoCodingAPI = GeoCodingAPI()
        if value == '':
            return ''
        else :
            geoCoding_pickup = geoCodingAPI.get_geocoding(value)
            if geoCoding_pickup["status"] == "OK" :
                return geoCoding_pickup['results'][0]['formatted_address']
            else:
                raise ValueError(f"Invalid pick-up location: {value}")
    @field_validator('destination_location')
    @classmethod
    def validate_destination(cls, value : str, info: ValidationInfo):
        geoCodingAPI = GeoCodingAPI()
        
        # print (geoCoding_destination['results'][0]['formatted_address'])
        if value == '':
            return ''
        else :
            geoCoding_destination = geoCodingAPI.get_geocoding(value)
            if geoCoding_destination["status"] == "OK":
                if geoCoding_destination['results'][0]['formatted_address'] == info.data['pick_up_location']:
                    raise ValueError(f"Invalid destination location: {value}")
                else:
                    return geoCoding_destination['results'][0]['formatted_address']
            else:
            
                raise ValueError(f"Invalid destination location: {value}")

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    # quote_id: str
    # booking_info: BookingCarDetails
    
def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_personal_details.model_dump().items():
        if value in [None, "", 0]:  # Add other 'empty' conditions if needed
            print(f"Field '{field}' is empty.")
            ask_for.append(field)
    return ask_for


def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    ic(updated_details)
    return updated_details

def ask_for_info(ask_list: list):
    first_prompt = ChatPromptTemplate.from_template(
        """Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. 
        Explain you're gathering info to help.
        ### ask_for list: {ask_for}"""
    )

    info_gathering_chain = first_prompt | llm | StrOutputParser()
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_list})
    print(first_prompt)
    return ai_chat
def filter_response(text_input, user_details : BookingCarDetails ):
    chain = llm.with_structured_output(BookingCarDetails)
    res = chain.invoke(text_input)
    user_details = add_non_empty_details(user_details,res)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for

In [ ]:
def change_info_field(fields : List[str], booking_details : BookingCarDetails):
    for field in fields:
        ai_response = ask_for_info([field])
        print(ai_response)
        text_input = input()
        user_details, ask_for = filter_response(text_input, booking_details)

    print(user_details)

In [ ]:
def get_booking_details():
    """ Call function to get the details for a booking from user"""
    chain = llm.with_structured_output(BookingCarDetails)
    msg = 'i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow, my name is Huy call me 0917181880'
    response =chain.invoke(msg)
    booking_details = BookingCarDetails(
        name="", number_phone="", pick_up_location="", destination_location="", pick_up_time=""
    )
    user_details= add_non_empty_details(booking_details, response)
    ask_for = check_what_is_empty(user_details)
    while ask_for:  
        ai_response = ask_for_info(ask_for)
        print(ai_response)
        text_input = input()
        user_details, ask_for = filter_response(text_input, user_details)
        print(ask_for)
    return user_details

In [ ]:
user_details = get_booking_details()

ic| updated_details: BookingCarDetails(name='Huy', number_phone='0917181880', pick_up_location='460 Đường Trần Đại Nghĩa, Hoà Hải, Ngũ Hành Sơn, Đà Nẵng 550000, Vietnam', destination_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='9 tomorrow')


In [ ]:
user_details = BookingCarDetails(name="hUY",number_phone="",pick_up_location="271 Nguyen Van Linh, Da Nang", destination_location="01 Nguyen Van Linh, Da Nang", pick_up_time="now")

In [ ]:
ai_response = ask_for_info(['name'])

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]


In [ ]:
print(ai_response)

I'm gathering some information to help you better. Could you please tell me your name?


In [ ]:
user_details = update_details(user_details, 'name', 'khoa')

In [ ]:
chain = llm.with_structured_output(BookingCarDetails)
msg = 'bye'
response =chain.invoke(msg)

In [ ]:
response

BookingCarDetails(name='John Doe', number_phone='123-456-7890', pick_up_location='123 Main St, Springfield, MA 01105, USA', destination_location='456 Elm St, West Springfield, MA 01089, USA', pick_up_time='3:00 PM')

In [ ]:
def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details

In [ ]:
user_details = add_non_empty_details( user_details,response)

ic| updated_details: BookingCarDetails(name='Khoa', number_phone='', pick_up_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='HAGL Plaza Danang, 01 Đ. Nguyễn Văn Linh, Nam Dương, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='now')
ic| non_empty_details: {'name': 'Khoa'}


In [ ]:

details, ask_for = filter_response("0917181880", user_details )

ic| updated_details: BookingCarDetails(name='khoa', number_phone='0917181880', pick_up_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='HAGL Plaza Danang, 01 Đ. Nguyễn Văn Linh, Nam Dương, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='now')


In [ ]:
print(user_details)

name='Khoa' number_phone='' pick_up_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam' destination_location='HAGL Plaza Danang, 01 Đ. Nguyễn Văn Linh, Nam Dương, Hải Châu, Đà Nẵng 550000, Vietnam' pick_up_time='now'


In [ ]:
change_info_field(['destination_location'],user_details)

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering some information to help you better. Could you please tell me your destination location?
name='Nguyen Van Linh' number_phone='0917181880' pick_up_location='1 Đ. Nguyễn Văn Linh, Phường 7, Bình Chánh, Hồ Chí Minh, Vietnam' destination_location={'271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam'} pick_up_time='9 tomorrow'


c:\Users\OS\anaconda3\envs\env_langchain\Lib\site-packages\pydantic\main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `set` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [ ]:
user_details

BookingCarDetails(name='hUY', number_phone='', pick_up_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='HAGL Plaza Danang, 01 Đ. Nguyễn Văn Linh, Nam Dương, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='now')

In [ ]:
def change_info_field(fields : List[str], booking_details : BookingCarDetails):
    for field in fields:
        ai_response = ask_for_info([field])
        print(ai_response)
        text_input = input()
        chain = llm.with_structured_output(BookingCarDetails)
        response =chain.invoke(text_input)
        booking_details = add_non_empty_details( booking_details,response)
    print(booking_details)

In [ ]:
change_info_field(['name', 'pick_up_location'], user_details)

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering some information to help you better. Could you please tell me your name?


ic| updated_details: BookingCarDetails(name='Khoa', number_phone='', pick_up_location='271 Đ. Nguyễn Văn Linh, Vĩnh Trung, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='HAGL Plaza Danang, 01 Đ. Nguyễn Văn Linh, Nam Dương, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='now')
ic| non_empty_details: {'name': 'Khoa'}


input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering some information to help you better. Could you please provide the pick-up location?


ic| updated_details: BookingCarDetails(name='John Doe', number_phone='1234567890', pick_up_location='5 Đ. Nguyễn Văn Linh, Bình Hiên, Hải Châu, Đà Nẵng 550000, Vietnam', destination_location='Danang International Airport, Đ. Nguyễn Văn Linh, Hải Châu, Đà Nẵng 550000, Vietnam', pick_up_time='2023-10-15 10:00 AM')
ic| non_empty_details: {'destination_location': 'Danang International Airport, Đ. Nguyễn Văn Linh, '
                                                'Hải Châu, Đà Nẵng 550000, Vietnam',
                        'name': 'John Doe',
                        'number_phone': '1234567890',
                        'pick_up_location': '5 Đ. Nguyễn Văn Linh, Bình Hiên, Hải Châu, Đà Nẵng '
                                            '550000, Vietnam',
                        'pick_up_time': '2023-10-15 10:00 AM'}


name='John Doe' number_phone='1234567890' pick_up_location='5 Đ. Nguyễn Văn Linh, Bình Hiên, Hải Châu, Đà Nẵng 550000, Vietnam' destination_location='Danang International Airport, Đ. Nguyễn Văn Linh, Hải Châu, Đà Nẵng 550000, Vietnam' pick_up_time='2023-10-15 10:00 AM'
